In [40]:
# Imports 
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from pycocotools.coco import COCO
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch.optim as optim
import random
from PIL import Image
from matplotlib import patches
from torchvision.transforms import functional as F

In [21]:
# Loading the Faster RCNN model
# Script to load the Modal 
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
num_classes = 9 + 1 # COCO class IDs + background class
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model_save_path = "faster_rcnn.pth"
model.load_state_dict(torch.load(model_save_path))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

# Open the csv file, get all the image name and let the faster RCNN predict what the class it is and stuff
# Create a coco dataset with all the images and bounding box
# Use the images and bounding box to predict the lat lon for different model

In [43]:
import os
import pandas as pd
import math

# Function to open the CSV file, correct the image names, and check if they exist in the folder
def open_csv(filepath):
    """Opens CSV, returns modified filenames, and checks if they exist in the folder."""
    df = pd.read_csv(filepath, encoding='utf-8')  # Ensure correct encoding
    
    corrected_image = []

    # Making the correct format
    for image in df["img_name"]:
        if isinstance(image, str):  # Ensure the value is a string
            corrected_img_name = image.replace(":", "_").strip()  # Replace ":" and remove leading/trailing spaces
            corrected_image.append(corrected_img_name)
        else:
            continue
            
    for image in df["darker_img_name"]:
        if isinstance(image, str):  # Ensure the value is a string
            corrected_img_name = image.replace(":", "_").strip()  # Replace ":" and remove leading/trailing spaces
            corrected_image.append(corrected_img_name)
        else:
            continue

    for image in df["darkest_img_name"]:
        if isinstance(image, str):  # Ensure the value is a string
            corrected_img_name = image.replace(":", "_").strip()  # Replace ":" and remove leading/trailing spaces
            corrected_image.append(corrected_img_name)
        else:
            continue
            
    return corrected_image

# Function to check if the image exists in the folder
def check_if_image_exists_in_the_folder(folder_path, image_name):
    # Skip if image_name is NaN or not a string
    if isinstance(image_name, str):
        full_path = os.path.join(folder_path, image_name)
        return os.path.exists(full_path)
    else:
        return False  # If it's not a string, consider it not found

# Folder path where the images are stored
folder_path = "test_5/captured_images"

# Example usage
csv_file_path = "test_5/position_data_logger.csv"

# Load the image names and other data from the CSV
image_list, lat, lon = open_csv(csv_file_path)

# Create a new list for images found in the folder
found_images = []

# Loop through image names and check if they exist
for img in image_list:
    if check_if_image_exists_in_the_folder(folder_path, img):
        found_images.append(img)

# Print the list of found images
# print("Found images:", found_images)
print(f"No of Found Images: {len(found_images)}")
# Function to count the number of images in a folder
def count_images_in_folder(folder_path, extensions=[".jpg", ".jpeg", ".png", ".gif"]):
    """Counts the number of images in a folder based on the specified extensions."""
    image_count = 0

    # List all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file extension is in the list of valid image extensions
        if any(file_name.lower().endswith(ext) for ext in extensions):
            image_count += 1

    return image_count
# Get the number of images in the folder
num_images = count_images_in_folder(folder_path)

print(f"Number of images in the folder: {num_images}")


KeyError: 'latitude'

In [42]:
# Load the Faster RCNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1)

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one (for 9 classes + background)
num_classes = 9 + 1  # 9 classes + background
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the model state dict (from previously saved model)
model_save_path = "faster_rcnn.pth"
model.load_state_dict(torch.load(model_save_path))
model.eval()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Categories for COCO format
categories = [{"id": i, "name": f"class_{i}"} for i in range(1, 10)]  # Modify with actual category names if you have them

# Function to load and preprocess images
def load_image(file_path):
    image = Image.open(file_path).convert("RGB")
    return F.to_tensor(image).unsqueeze(0)  # Convert to tensor and add batch dimension

# Function to annotate images in COCO format
def annotate_images_in_folder(found_images, folder_path, output_json_path, threshold=0.5):
    image_id = 1
    annotation_id = 1
    coco_annotations = {
        "images": [],
        "annotations": [],
        "categories": categories
    }

    # Loop through found images, checking if they exist in the folder
    for filename in found_images:
        file_path = os.path.join(folder_path, filename)
        if not os.path.isfile(file_path):
            print(f"Image not found: {filename}")
            continue  # Skip if the image is not found

        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Load and preprocess the image
            image = load_image(file_path).to(device)

            # Predict using Faster R-CNN
            with torch.no_grad():
                prediction = model(image)

            # Get image dimensions
            image_np = image.squeeze().permute(1, 2, 0).cpu().numpy()
            height, width, _ = image_np.shape

            # Append image details to COCO annotations
            coco_annotations["images"].append({
                "id": image_id,
                "file_name": filename,
                "width": int(width),
                "height": int(height)
            })

            # Get the predictions
            boxes = prediction[0]['boxes'].cpu().numpy()
            scores = prediction[0]['scores'].cpu().numpy()
            labels = prediction[0]['labels'].cpu().numpy()

            # Create a list of (score, label, box) tuples and sort by score descending
            predictions = sorted(zip(scores, labels, boxes), key=lambda x: x[0], reverse=True)

            # Keep track of the count of predictions per label type (to limit to 2 per class)
            label_counts = {}

            # Add top 2 predictions for each class
            for score, label, box in predictions:
                if score >= threshold:
                    if label not in label_counts:
                        label_counts[label] = 0
                    if label_counts[label] < 2:
                        xmin, ymin, xmax, ymax = box
                        width = xmax - xmin
                        height = ymax - ymin
                        coco_annotations["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": int(label),
                            "bbox": [float(xmin), float(ymin), float(width), float(height)],
                            "score": float(score)
                        })
                        label_counts[label] += 1
                        annotation_id += 1

            image_id += 1

    # Save the annotations to a JSON file
    with open(output_json_path, 'w') as f:
        json.dump(coco_annotations, f, indent=4)

# Example usage:
folder_path = 'test_5/captured_images'  # The folder where the images are stored
output_json_path = 'test_5/2BBox_Each_Class_Annotation.json'

# Annotate images and create a COCO-style dataset
annotate_images_in_folder(found_images, folder_path, output_json_path)
print("Finished creating COCO-style annotations.")


Finished creating COCO-style annotations.
